In [1]:
pip install -U sentence-transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 86.0/86.0 KB 1.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.3/6.3 MB 49.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 190.3/190.3 KB 15.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 44.7 MB/s eta 0:00:00
  Created wheel for sentence-transformers: filename=sentence_transformers-2.2.2-py3-none-any.whl size=125938 sha256=3c2fa2353efb64acef7290ed64b5370b89ddb93323c74fcf3c653a13507b1f1f
  Stored in directory: /root/.cache/pip/wheels/5e/6f/8c/d88aec621f3f542d26fac0342bef5e693335d125f4e54aeffe
Successfully built sentence-transformers


In [2]:
!pip install "optimum[onnxruntime]==1.5.0" transformers evaluate mkl-include mkl --upgrade


Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 187.2/187.2 KB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 KB 7.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 27.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 254.4/254.4 MB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.0/46.0 KB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 11.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.5/13.5 MB 12.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 15.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 462.8/462.8 KB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.0/132.0 KB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.0/213.0 KB 14.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━

In [14]:
from sentence_transformers import SentenceTransformer
import time

sentence = 'Royal Bank of Canada is a Canadian multinational financial services company and the largest bank in Canada by market capitalization. The bank serves over 17 million clients and has more than 89,000 employees worldwide'

model = SentenceTransformer('sentence-transformers/all-distilroberta-v1')
v_st = time.time() * 1000
vanilla_emb = model.encode(sentence)
v_et = time.time() * 1000
print('time taken for vanilla : {}'.format(v_et - v_st))


time taken for vanilla : 132.3349609375


In [12]:
from transformers import Pipeline
import torch.nn.functional as F
import torch

# copied from the model card
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)


class SentenceEmbeddingPipeline(Pipeline):
    def _sanitize_parameters(self, **kwargs):
        # we don't have any hyperameters to sanitize
        preprocess_kwargs = {}
        return preprocess_kwargs, {}, {}

    def preprocess(self, inputs):
        encoded_inputs = self.tokenizer(inputs, padding=True, truncation=True, return_tensors='pt')
        return encoded_inputs

    def _forward(self, model_inputs):
        outputs = self.model(**model_inputs)
        return {"outputs": outputs, "attention_mask": model_inputs["attention_mask"]}

    def postprocess(self, model_outputs):
        # Perform pooling
        sentence_embeddings = mean_pooling(model_outputs["outputs"], model_outputs['attention_mask'])
        # Normalize embeddings
        sentence_embeddings = F.normalize(sentence_embeddings, p=2, dim=1)
        return sentence_embeddings


In [5]:
from optimum.onnxruntime import ORTModelForFeatureExtraction
from transformers import AutoTokenizer
from pathlib import Path

onnx_path = Path("onnx")

tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-distilroberta-v1')


/usr/local/lib/python3.8/dist-packages/transformers/generation_utils.py:24: FutureWarning: Importing `GenerationMixin` from `src/transformers/generation_utils.py` is deprecated and will be removed in Transformers v5. Import as `from transformers import GenerationMixin` instead.
  warnings.warn(


In [15]:
# load optimized model
model = ORTModelForFeatureExtraction.from_pretrained(onnx_path, file_name="model_optimized_quantized.onnx")

# create optimized pipeline
optimized_emb = SentenceEmbeddingPipeline(model=model, tokenizer=tokenizer)

o_st = time.time() * 1000
optimized_emb_pred = optimized_emb(sentence)
o_et =  time.time() * 1000

print('time taken for optimized quantized model: {}'.format(o_et - o_st))

time taken for optimized quantized model: 52.700439453125


In [16]:
optimized_emb_pred

tensor([[-3.7310e-02,  5.5734e-02,  9.1432e-03,  2.4154e-02, -2.0351e-02,
         -3.1886e-02,  3.0823e-03,  5.3715e-02, -2.6739e-02, -1.1252e-02,
         -2.5986e-02,  9.7549e-02, -1.7449e-02, -4.3857e-02, -7.5206e-02,
          2.2248e-02, -5.5003e-02,  1.7782e-02,  2.8467e-02, -2.8150e-02,
          1.8533e-02, -1.6071e-02, -1.8746e-02, -7.2122e-03, -2.7609e-02,
          6.9813e-03,  3.3660e-02,  1.0700e-02, -5.6438e-02,  7.8988e-02,
          1.2906e-02, -5.4309e-03,  1.4703e-02, -6.7639e-02,  3.8255e-02,
         -5.6964e-02, -5.1040e-03,  1.2940e-02,  5.4253e-02,  3.3922e-03,
          2.0413e-02, -7.8722e-03,  2.0498e-02, -1.5117e-02,  2.3007e-02,
          2.7469e-02, -1.7764e-02,  6.6413e-03, -6.3284e-03, -3.6325e-02,
         -2.9307e-02, -8.9882e-03,  9.2503e-03, -1.0362e-03,  3.9264e-02,
          6.7627e-04, -6.5784e-02,  6.5745e-03, -2.2015e-02,  9.6801e-03,
          4.4603e-02, -9.3227e-03, -2.0392e-02,  6.3533e-02, -1.3131e-02,
          1.7636e-02,  4.7145e-02, -3.

In [17]:
from sklearn.metrics.pairwise import cosine_similarity

In [18]:
cosine_similarity([vanilla_emb],optimized_emb_pred)

array([[0.98724216]], dtype=float32)